# What is a custom dataset?

Custom datasets are datasets not created specifically for use as input data for Neural Networks, so we need to adjust said data before we are able to use it to train and test our models.

We use different PyTorch domain libraries dependent on our problem space, these can include:

- Vision - `torchvision`
- Text - `torchtext`
- Audio - `torchaudio`
- Recommendation Systems - `torchrec`

TorchData is also a new domain we can use for all sort of data.

## FoodVisionMini

This will be a CNN that takes in custom data, following our PyTorch workflow as seen below.

- Get data ready (turn into tensors)
- Pick a loss function and optimizer
- Build or pick a pretrained model to suit our problem
- Fit the model to the data and make a prediction
- Evaluate the model
- Improve the model through experimentation
- Save and reload the trained model

In [2]:
import torch
from torch import nn

# Setup device-agnostic code

device = "cuda" if torch.cuda.is_available() else "cpu"

## 1. Get Data

Food101 starts with 101 different classes of food and 1000 images per class.

Our dataset, a subset of Food101, starts with just 3 classes, and 100 images per class.

When starting out ML projects, it's important to try things on a small scale and then increase the scale when necessary. The use is to speed up how fast you can experiment. If we don't do this, we will take ages to train every model.

In [8]:
import requests
import zipfile
from pathlib import Path


# Setup path to a data folder
data_path = Path("data/")
image_path = data_path / "pizza_steak_sushi"

if image_path.is_dir():
    print(f"{image_path} directory already exists... skipping download")
else:
    print(f"{image_path} does not exist, creating one...")
    image_path.mkdir(parents=True, exist_ok=True)

# Download data to pizza_steak_sushi

with open(data_path / "pizza_steak_sushi.zip", "wb") as f:
    request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
    print("Downloading data...")
    f.write(request.content)

# Unzip file
with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_ref:
    print("Unzipping data")
    zip_ref.extractall(image_path)

data/pizza_steak_sushi directory already exists... skipping download
Unzipping data


## 2. Preparing and exploring the dataset

In [10]:
import os

def walk_through_dir(dir_path):
    for dirpath, dirnames, filenames in os.walk(dir_path):
        print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

walk_through_dir(image_path)


There are 2 directories and 0 images in 'data/pizza_steak_sushi'.
There are 3 directories and 0 images in 'data/pizza_steak_sushi/test'.
There are 0 directories and 19 images in 'data/pizza_steak_sushi/test/steak'.
There are 0 directories and 31 images in 'data/pizza_steak_sushi/test/sushi'.
There are 0 directories and 25 images in 'data/pizza_steak_sushi/test/pizza'.
There are 3 directories and 0 images in 'data/pizza_steak_sushi/train'.
There are 0 directories and 75 images in 'data/pizza_steak_sushi/train/steak'.
There are 0 directories and 72 images in 'data/pizza_steak_sushi/train/sushi'.
There are 0 directories and 78 images in 'data/pizza_steak_sushi/train/pizza'.


In [11]:
# Setup train and test paths

train_dir = image_path / "train"
test_dir = image_path / "test"

# We can use this as an alternative to the train test split we did earlier

### 2.1 Visualising and image 